# All material ©2019, Alex Siegman

---

## Welcome to Projects in Programming & Data Science. Today we're going to jump right in to the mix and leverage the CitiBike API to populate a sqlite database at regular intervals. Consider this your warm-up for the semester! 

---

In [8]:
import sqlite3

In [9]:
con = sqlite3.connect('citibikeData.db') # create our database

## Now, let's check out the API we'll be working with:

### https://streamdata.io/developers/api-gallery/new-york-citibike-api/

#### First, we'll request the json from the CitiBike API URL and just print it out to get a quick glimpse

In [10]:
import json 
import urllib.request # https://docs.python.org/3/library/urllib.request.html

with urllib.request.urlopen("https://feeds.citibikenyc.com/stations/stations.json") as url:
    data = json.loads(url.read().decode())
    print(data)

{'executionTime': '2019-09-06 03:36:44 PM', 'stationBeanList': [{'id': 168, 'stationName': 'W 18 St & 6 Ave', 'availableDocks': 35, 'totalDocks': 47, 'latitude': 40.73971301, 'longitude': -73.99456405, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 11, 'stAddress1': 'W 18 St & 6 Ave', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2019-09-06 03:34:46 PM', 'landMark': ''}, {'id': 281, 'stationName': 'Grand Army Plaza & Central Park S', 'availableDocks': 46, 'totalDocks': 66, 'latitude': 40.7643971, 'longitude': -73.97371465, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 18, 'stAddress1': 'Grand Army Plaza & Central Park S', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2019-09-06 03:32:58 PM', 'landMark': ''}, {'id': 285, 'stationName': 'Broadway & E 14 St', 'availableDocks': 39, 'totalDocks': 

In [11]:
stations = data['stationBeanList'] # iterate through the json to find the station data

In [12]:
import pandas as pd # we'll use pandas just to visualize our data, NOT to query it

df_stations = pd.DataFrame(stations)
df_stations.head() # check the first five station entries

# you'll note that it has set 'altitude' as our index; if we were working only in Pandas we'd probably want to 
# change that, but we can leave it alone for now

altitude  availableBikes  availableDocks city   id landMark  \
0                       11              35       168            
1                       18              46       281            
2                       13              39       285            
3                        6              27       298            
4                        5              28       304            

    lastCommunicationTime   latitude location  longitude postalCode  \
0  2019-09-06 03:34:46 PM  40.739713          -73.994564              
1  2019-09-06 03:32:58 PM  40.764397          -73.973715              
2  2019-09-06 03:33:18 PM  40.734546          -73.990741              
3  2019-09-06 03:34:52 PM  40.686832          -73.979677              
4  2019-09-06 03:35:23 PM  40.704633          -74.013617              

                          stAddress1 stAddress2  \
0                    W 18 St & 6 Ave              
1  Grand Army Plaza & Central Park S              
2                 Broadway & E 14 St              
3            3 Ave & Schermerhorn St              
4              Broadway & Battery Pl              

                         stationName  statusKey statusValue  testStation  \
0                    W 18 St & 6 Ave          1  In Service        False   
1  Grand Army Plaza & Central Park S          1  In Service        False   
2                 Broadway & E 14 St          1  In Service        False   
3            3 Ave & Schermerhorn St          1  In Service        False   
4              Broadway & Battery Pl          1  In Service        False   

   totalDocks  
0          47  
1          66  
2          53  
3          35  
4          33

In [13]:
sql = "CREATE TABLE IF NOT EXISTS StationsData (station_id int, stationName varchar(250), availableDocks int, totalDocks int, latitude float, longitude float, statusValue varchar(250), statusKey int, availableBikes int, stAddress1 varchar(250), stAddress2 varchar(250), city varchar(250), postalCode varchar(250), location varchar(250), altitude varchar(250), testStation bool, lastCommunicationTime date, landMark varchar(250));" 

con.execute(sql)
con.commit()

In [14]:
from datetime import datetime # import the datetime library 

In [18]:
query_template = """INSERT OR IGNORE INTO StationsData(station_id, stationName, availableDocks, totalDocks, latitude, \
longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, \
testStation, lastCommunicationTime, landMark) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"""

for entry in stations: # for every station entry in the json 
    station_id = int(entry['id']) # find and set station_id
    stationName = str(entry['stationName'])
    availableDocks = int(entry['availableDocks'])
    totalDocks = int(entry['totalDocks'])
    latitude = str(entry['latitude'])
    longitude = str(entry['longitude'])
    statusValue = str(entry['statusValue'])
    statusKey = int(entry['statusKey'])
    availableBikes = int(entry['availableBikes'])
    stAddress1 = str(entry['stAddress1'])
    stAddress2 = str(entry['stAddress2'])
    city = str(entry['city'])
    postalCode = str(entry['postalCode'])
    location = str(entry['location'])
    altitude = str(entry['altitude'])
    testStation = bool(entry['testStation'])
    lastCommunicationTime = entry['lastCommunicationTime']
    landMark = str(entry['landMark'])
                           
    print("Inserting Station:", station_id, stationName, availableDocks, totalDocks, latitude, longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, testStation, lastCommunicationTime, landMark) 
    
    query_parameters = (station_id, stationName, availableDocks, totalDocks, latitude, longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, testStation, lastCommunicationTime, landMark) 
    
    con.execute(query_template, query_parameters)
    
con.commit()

Inserting Station: 168 W 18 St & 6 Ave 35 47 40.73971301 -73.99456405 In Service 1 11 W 18 St & 6 Ave      False 2019-09-06 03:34:46 PM 
Inserting Station: 281 Grand Army Plaza & Central Park S 46 66 40.7643971 -73.97371465 In Service 1 18 Grand Army Plaza & Central Park S      False 2019-09-06 03:32:58 PM 
Inserting Station: 285 Broadway & E 14 St 39 53 40.73454567 -73.99074142 In Service 1 13 Broadway & E 14 St      False 2019-09-06 03:33:18 PM 
Inserting Station: 298 3 Ave & Schermerhorn St 27 35 40.68683208 -73.9796772 In Service 1 6 3 Ave & Schermerhorn St      False 2019-09-06 03:34:52 PM 
Inserting Station: 304 Broadway & Battery Pl 28 33 40.70463334 -74.01361706 In Service 1 5 Broadway & Battery Pl      False 2019-09-06 03:35:23 PM 
Inserting Station: 307 Canal St & Rutgers St 18 30 40.71427487 -73.98990025 In Service 1 11 Canal St & Rutgers St      False 2019-09-06 03:36:27 PM 
Inserting Station: 337 Old Slip & Front St 28 37 40.7037992 -74.00838676 In Service 1 9 Old Slip & F

Inserting Station: 319 Fulton St & Broadway 4 33 40.711066 -74.009447 In Service 1 28 Fulton St & Broadway      False 2019-09-06 03:34:25 PM 
Inserting Station: 320 Leonard St & Church St 1 27 40.717571 -74.005549 In Service 1 24 Leonard St & Church St      False 2019-09-06 03:36:01 PM 
Inserting Station: 321 Cadman Plaza E & Red Cross Pl 1 27 40.69991755 -73.98971773 In Service 1 25 Cadman Plaza E & Red Cross Pl      False 2019-09-06 03:35:02 PM 
Inserting Station: 323 Lawrence St & Willoughby St 1 39 40.69236178 -73.98631746 In Service 1 38 Lawrence St & Willoughby St      False 2019-09-06 03:33:30 PM 
Inserting Station: 324 DeKalb Ave & Hudson Ave 4 51 40.689888 -73.981013 In Service 1 46 DeKalb Ave & Hudson Ave      False 2019-09-06 03:36:38 PM 
Inserting Station: 325 E 19 St & 3 Ave 28 35 40.73624527 -73.98473765 In Service 1 6 E 19 St & 3 Ave      False 2019-09-06 03:33:26 PM 
Inserting Station: 326 E 11 St & 1 Ave 34 41 40.72953837 -73.98426726 In Service 1 6 E 11 St & 1 Ave    

Inserting Station: 501 FDR Drive & E 35 St 69 79 40.744219 -73.97121214 In Service 1 9 FDR Drive & E 35 St      False 2019-09-06 03:36:43 PM 
Inserting Station: 502 Henry St & Grand St 26 30 40.714215 -73.981346 In Service 1 4 Henry St & Grand St      False 2019-09-06 03:32:46 PM 
Inserting Station: 503 E 20 St & Park Ave 0 31 40.73827428 -73.98751968 In Service 1 30 E 20 St & Park Ave      False 2019-09-06 03:36:19 PM 
Inserting Station: 504 1 Ave & E 16 St 48 53 40.73221853 -73.98165557 In Service 1 3 1 Ave & E 16 St      False 2019-09-06 03:33:51 PM 
Inserting Station: 505 6 Ave & W 33 St 23 39 40.74901271 -73.98848395 In Service 1 13 6 Ave & W 33 St      False 2019-09-06 03:35:40 PM 
Inserting Station: 507 E 25 St & 2 Ave 27 45 40.73912601 -73.97973776 In Service 1 16 E 25 St & 2 Ave      False 2019-09-06 03:36:22 PM 
Inserting Station: 508 W 46 St & 11 Ave 21 25 40.76341379 -73.99667444 In Service 1 3 W 46 St & 11 Ave      False 2019-09-06 03:34:21 PM 
Inserting Station: 509 9 Ave

Inserting Station: 3162 W 78 St & Broadway 35 39 40.78339981 -73.98093133 In Service 1 3 W 78 St & Broadway      False 2019-09-06 03:34:48 PM 
Inserting Station: 3163 Central Park West & W 68 St 40 43 40.7734066 -73.97782542 In Service 1 3 Central Park West & W 68 St      False 2019-09-06 03:32:58 PM 
Inserting Station: 3164 Columbus Ave & W 72 St 64 67 40.7770575 -73.97898475 In Service 1 0 Columbus Ave & W 72 St      False 2019-09-06 03:33:55 PM 
Inserting Station: 3165 Central Park West & W 72 St 20 51 40.77579376683666 -73.9762057363987 In Service 1 29 Central Park West & W 72 St      False 2019-09-06 03:34:41 PM 
Inserting Station: 3166 Riverside Dr & W 72 St 38 41 40.78057799010334 -73.98562431335449 In Service 1 1 Riverside Dr & W 72 St      False 2019-09-06 03:33:38 PM 
Inserting Station: 3167 Amsterdam Ave & W 73 St 50 55 40.77966809007312 -73.98093044757842 In Service 1 5 Amsterdam Ave & W 73 St      False 2019-09-06 03:34:02 PM 
Inserting Station: 3168 Central Park West & W 

Inserting Station: 3360 Amsterdam Ave & W 79 St 24 25 40.7829391 -73.9786517 In Service 1 1 Amsterdam Ave & W 79 St      False 2019-09-06 03:36:06 PM 
Inserting Station: 3361 Carroll St & 6 Ave 7 7 40.6740886 -73.9787282 Not In Service 3 0 Carroll St & 6 Ave      False 2019-08-12 12:57:20 PM 
Inserting Station: 3362 Madison Ave & E 82 St 26 34 40.7781314 -73.96069399 In Service 1 8 Madison Ave & E 82 St      False 2019-09-06 03:36:22 PM 
Inserting Station: 3363 E 102 St & Park Ave 11 33 40.7904828 -73.95033068 In Service 1 21 E 102 St & Park Ave      False 2019-09-06 03:32:56 PM 
Inserting Station: 3364 Carroll St & 5 Ave 20 27 40.6751622 -73.9814832 In Service 1 7 Carroll St & 5 Ave      False 2019-09-06 03:36:34 PM 
Inserting Station: 3365 3 St & 7 Ave 23 27 40.6703837 -73.97839676 In Service 1 3 3 St & 7 Ave      False 2019-09-06 03:32:51 PM 
Inserting Station: 3366 West End Ave & W 107 St 30 31 40.8021174 -73.9681805305 In Service 1 1 West End Ave & W 107 St      False 2019-09-06 0

Inserting Station: 3537 Carlton Ave & Dean St 28 29 40.6809741 -73.9710097 In Service 1 1 Carlton Ave & Dean St      False 2019-09-06 03:33:47 PM 
Inserting Station: 3538 W 110 St & Amsterdam Ave 42 45 40.802692 -73.96295 In Service 1 3 W 110 St & Amsterdam Ave      False 2019-09-06 03:34:48 PM 
Inserting Station: 3539 W 116 St & Amsterdam Ave 20 43 40.8067581 -73.9607082 In Service 1 23 W 116 St & Amsterdam Ave      False 2019-09-06 03:34:18 PM 
Inserting Station: 3540 Morningside Ave & W 123 St 20 35 40.81 -73.9551508 In Service 1 14 Morningside Ave & W 123 St      False 2019-09-06 03:33:36 PM 
Inserting Station: 3541 Amsterdam Ave & W 125 St 1 38 40.813358 -73.956461 In Service 1 37 Amsterdam Ave & W 125 St      False 2019-09-06 03:35:47 PM 
Inserting Station: 3542 Amsterdam Ave & W 119 St 21 29 40.8086249 -73.9596214 In Service 1 7 Amsterdam Ave & W 119 St      False 2019-09-06 03:33:53 PM 
Inserting Station: 3543 Morningside Dr & Amsterdam Ave 35 35 40.81028506306003 -73.957364559

Inserting Station: 3638 Washington St 0 18 40.7242941 -74.0354826 In Service 1 18 Washington St      False 2019-09-06 03:36:34 PM 
Inserting Station: 3639 Harborside 8 30 40.7192517 -74.034234 In Service 1 21 Harborside      False 2019-09-06 03:33:23 PM 
Inserting Station: 3640 Journal Square 3 18 40.73367 -74.0625 In Service 1 15 Journal Square      False 2019-09-06 03:36:29 PM 
Inserting Station: 3641 Broadway & W 25 St 12 54 40.74286877312112 -73.98918628692627 In Service 1 41 Broadway & W 25 St      False 2019-09-06 03:33:31 PM 
Inserting Station: 3646 35 Ave & 10 St 10 26 40.76315481910373 -73.93985509872437 In Service 1 16 35 Ave & 10 St      False 2019-09-06 03:34:02 PM 
Inserting Station: 3647 48 Ave & 30 Pl 4 23 40.74128309355993 -73.93725872039794 In Service 1 19 48 Ave & 30 Pl      False 2019-09-06 03:33:36 PM 
Inserting Station: 3648 Flushing Ave & Vanderbilt Ave 3 25 40.697950315695444 -73.97077560424805 In Service 1 21 Flushing Ave & Vanderbilt Ave      False 2019-09-06 0

Inserting Station: 3774 Knickerbocker Ave & George St 4 23 40.704425 -73.92836 In Service 1 19 Knickerbocker Ave & George St      False 2019-09-06 03:33:15 PM 
Inserting Station: 3775 Suydam St & Knickerbocker Ave 8 27 40.702013 -73.923769 In Service 1 19 Suydam St & Knickerbocker Ave      False 2019-09-06 03:34:36 PM 
Inserting Station: 3776 Central Ave & Starr Street 15 23 40.700003 -73.92834 In Service 1 8 Central Ave & Starr Street      False 2019-09-06 03:34:55 PM 
Inserting Station: 3777 Stockholm St & Wilson Ave 0 23 40.699304 -73.923044 In Service 1 23 Stockholm St & Wilson Ave      False 2019-09-06 03:33:59 PM 
Inserting Station: 3778 Irving Ave & DeKalb Ave 6 27 40.7027 -73.92095 In Service 1 21 Irving Ave & DeKalb Ave      False 2019-09-06 03:33:08 PM 
Inserting Station: 3779 Hart St & Wyckoff Ave 5 24 40.704876 -73.919911 In Service 1 18 Hart St & Wyckoff Ave      False 2019-09-06 03:35:09 PM 
Inserting Station: 3781 Greene Av & Myrtle Av 5 30 40.698568 -73.918877 In Servic

In [21]:
check = pd.read_sql("SELECT * FROM StationsData LIMIT 5", con=con)
check

station_id                        stationName  availableDocks  totalDocks  \
0         168                    W 18 St & 6 Ave              35          47   
1         281  Grand Army Plaza & Central Park S              46          66   
2         285                 Broadway & E 14 St              39          53   
3         298            3 Ave & Schermerhorn St              27          35   
4         304              Broadway & Battery Pl              28          33   

    latitude  longitude statusValue  statusKey  availableBikes  \
0  40.739713 -73.994564  In Service          1              11   
1  40.764397 -73.973715  In Service          1              18   
2  40.734546 -73.990741  In Service          1              13   
3  40.686832 -73.979677  In Service          1               6   
4  40.704633 -74.013617  In Service          1               5   

                          stAddress1 stAddress2 city postalCode location  \
0                    W 18 St & 6 Ave                                       
1  Grand Army Plaza & Central Park S                                       
2                 Broadway & E 14 St                                       
3            3 Ave & Schermerhorn St                                       
4              Broadway & Battery Pl                                       

  altitude  testStation   lastCommunicationTime landMark  
0                     0  2019-09-06 03:34:46 PM           
1                     0  2019-09-06 03:32:58 PM           
2                     0  2019-09-06 03:33:18 PM           
3                     0  2019-09-06 03:34:52 PM           
4                     0  2019-09-06 03:35:23 PM

## Last but not least, let's set things up so that our database automatically updates every 15 seconds. 

In [22]:
import time 
import json 
import urllib.request
from datetime import datetime

while True:
    
    with urllib.request.urlopen("https://feeds.citibikenyc.com/stations/stations.json") as url:
        data = json.loads(url.read().decode())

    stations = data['stationBeanList'] # iterate through the json to find the station data

    con = sqlite3.connect('citibikeData.db') # connect to our db

    query_template = """INSERT OR IGNORE INTO StationsData(station_id, stationName, availableDocks, totalDocks, latitude, \
    longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, \
    testStation, lastCommunicationTime, landMark) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"""

    for entry in stations: # for every station entry in the json 
        station_id = int(entry['id']) # find and set station_id
        stationName = str(entry['stationName'])
        availableDocks = int(entry['availableDocks'])
        totalDocks = int(entry['totalDocks'])
        latitude = str(entry['latitude'])
        longitude = str(entry['longitude'])
        statusValue = str(entry['statusValue'])
        statusKey = int(entry['statusKey'])
        availableBikes = int(entry['availableBikes'])
        stAddress1 = str(entry['stAddress1'])
        stAddress2 = str(entry['stAddress2'])
        city = str(entry['city'])
        postalCode = str(entry['postalCode'])
        location = str(entry['location'])
        altitude = str(entry['altitude'])
        testStation = bool(entry['testStation'])
        lastCommunicationTime = entry['lastCommunicationTime']
        landMark = str(entry['landMark'])
                           
        print("Inserting Station:", station_id, stationName, availableDocks, totalDocks, latitude, longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, testStation, lastCommunicationTime, landMark) 
        query_parameters = (station_id, stationName, availableDocks, totalDocks, latitude, longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, testStation, lastCommunicationTime, landMark) 
       
        con.execute(query_template, query_parameters)
    
    con.commit()
        
    time.sleep(15)

Inserting Station: 168 W 18 St & 6 Ave 35 47 40.73971301 -73.99456405 In Service 1 11 W 18 St & 6 Ave      False 2019-09-06 03:41:47 PM 
Inserting Station: 281 Grand Army Plaza & Central Park S 46 66 40.7643971 -73.97371465 In Service 1 18 Grand Army Plaza & Central Park S      False 2019-09-06 03:40:56 PM 
Inserting Station: 285 Broadway & E 14 St 39 53 40.73454567 -73.99074142 In Service 1 13 Broadway & E 14 St      False 2019-09-06 03:41:30 PM 
Inserting Station: 298 3 Ave & Schermerhorn St 27 35 40.68683208 -73.9796772 In Service 1 6 3 Ave & Schermerhorn St      False 2019-09-06 03:38:51 PM 
Inserting Station: 304 Broadway & Battery Pl 28 33 40.70463334 -74.01361706 In Service 1 5 Broadway & Battery Pl      False 2019-09-06 03:39:22 PM 
Inserting Station: 307 Canal St & Rutgers St 18 30 40.71427487 -73.98990025 In Service 1 11 Canal St & Rutgers St      False 2019-09-06 03:40:26 PM 
Inserting Station: 337 Old Slip & Front St 29 37 40.7037992 -74.00838676 In Service 1 8 Old Slip & F

Inserting Station: 361 Allen St & Hester St 35 43 40.71605866 -73.99190759 In Service 1 8 Allen St & Hester St      False 2019-09-06 03:42:08 PM 
Inserting Station: 362 Broadway & W 37 St 37 57 40.75172632 -73.98753523 In Service 1 20 Broadway & W 37 St      False 2019-09-06 03:38:56 PM 
Inserting Station: 363 West Thames St 35 49 40.70834698 -74.01713445 In Service 1 13 West Thames St      False 2019-09-06 03:41:02 PM 
Inserting Station: 364 Lafayette Ave & Classon Ave 26 27 40.68900443 -73.96023854 In Service 1 1 Lafayette Ave & Classon Ave      False 2019-09-06 03:41:47 PM 
Inserting Station: 366 Clinton Ave & Myrtle Ave 26 33 40.693261 -73.968896 In Service 1 7 Clinton Ave & Myrtle Ave      False 2019-09-06 03:40:02 PM 
Inserting Station: 368 Carmine St & 6 Ave 10 39 40.73038599 -74.00214988 In Service 1 28 Carmine St & 6 Ave      False 2019-09-06 03:40:28 PM 
Inserting Station: 369 Washington Pl & 6 Ave 15 21 40.73224119 -74.00026394 In Service 1 6 Washington Pl & 6 Ave      False

Inserting Station: 523 W 38 St & 8 Ave 40 51 40.75466591 -73.99138152 In Service 1 10 W 38 St & 8 Ave      False 2019-09-06 03:41:44 PM 
Inserting Station: 525 W 34 St & 11 Ave 6 56 40.75594159 -74.0021163 In Service 1 48 W 34 St & 11 Ave      False 2019-09-06 03:40:41 PM 
Inserting Station: 526 E 33 St & 5 Ave 14 40 40.74765947 -73.98490707 In Service 1 24 E 33 St & 5 Ave      False 2019-09-06 03:42:05 PM 
Inserting Station: 528 2 Ave & E 31 St 20 39 40.74290902 -73.97706058 In Service 1 19 2 Ave & E 31 St      False 2019-09-06 03:41:38 PM 
Inserting Station: 529 W 42 St & 8 Ave 18 41 40.7575699 -73.99098507 In Service 1 22 W 42 St & 8 Ave      False 2019-09-06 03:42:31 PM 
Inserting Station: 530 11 Ave & W 59 St 22 24 40.77149671054441 -73.99046033620834 In Service 1 0 11 Ave & W 59 St      False 2019-09-06 03:40:01 PM 
Inserting Station: 531 Forsyth St & Broome St 23 39 40.71893904 -73.99266288 In Service 1 15 Forsyth St & Broome St      False 2019-09-06 03:42:08 PM 
Inserting Stati

Inserting Station: 3162 W 78 St & Broadway 35 39 40.78339981 -73.98093133 In Service 1 3 W 78 St & Broadway      False 2019-09-06 03:38:46 PM 
Inserting Station: 3163 Central Park West & W 68 St 41 43 40.7734066 -73.97782542 In Service 1 2 Central Park West & W 68 St      False 2019-09-06 03:41:16 PM 
Inserting Station: 3164 Columbus Ave & W 72 St 64 67 40.7770575 -73.97898475 In Service 1 0 Columbus Ave & W 72 St      False 2019-09-06 03:41:53 PM 
Inserting Station: 3165 Central Park West & W 72 St 19 51 40.77579376683666 -73.9762057363987 In Service 1 30 Central Park West & W 72 St      False 2019-09-06 03:39:37 PM 
Inserting Station: 3166 Riverside Dr & W 72 St 38 41 40.78057799010334 -73.98562431335449 In Service 1 1 Riverside Dr & W 72 St      False 2019-09-06 03:41:35 PM 
Inserting Station: 3167 Amsterdam Ave & W 73 St 50 55 40.77966809007312 -73.98093044757842 In Service 1 5 Amsterdam Ave & W 73 St      False 2019-09-06 03:42:09 PM 
Inserting Station: 3168 Central Park West & W 

Inserting Station: 3359 E 68 St & Madison Ave 29 35 40.7691572 -73.96703464 In Service 1 6 E 68 St & Madison Ave      False 2019-09-06 03:42:33 PM 
Inserting Station: 3360 Amsterdam Ave & W 79 St 24 25 40.7829391 -73.9786517 In Service 1 1 Amsterdam Ave & W 79 St      False 2019-09-06 03:40:05 PM 
Inserting Station: 3361 Carroll St & 6 Ave 7 7 40.6740886 -73.9787282 Not In Service 3 0 Carroll St & 6 Ave      False 2019-08-12 12:57:20 PM 
Inserting Station: 3362 Madison Ave & E 82 St 26 34 40.7781314 -73.96069399 In Service 1 8 Madison Ave & E 82 St      False 2019-09-06 03:40:21 PM 
Inserting Station: 3363 E 102 St & Park Ave 11 33 40.7904828 -73.95033068 In Service 1 21 E 102 St & Park Ave      False 2019-09-06 03:40:54 PM 
Inserting Station: 3364 Carroll St & 5 Ave 20 27 40.6751622 -73.9814832 In Service 1 7 Carroll St & 5 Ave      False 2019-09-06 03:40:32 PM 
Inserting Station: 3365 3 St & 7 Ave 23 27 40.6703837 -73.97839676 In Service 1 4 3 St & 7 Ave      False 2019-09-06 03:41:2

Inserting Station: 3538 W 110 St & Amsterdam Ave 42 45 40.802692 -73.96295 In Service 1 3 W 110 St & Amsterdam Ave      False 2019-09-06 03:38:47 PM 
Inserting Station: 3539 W 116 St & Amsterdam Ave 20 43 40.8067581 -73.9607082 In Service 1 23 W 116 St & Amsterdam Ave      False 2019-09-06 03:42:16 PM 
Inserting Station: 3540 Morningside Ave & W 123 St 20 35 40.81 -73.9551508 In Service 1 14 Morningside Ave & W 123 St      False 2019-09-06 03:41:33 PM 
Inserting Station: 3541 Amsterdam Ave & W 125 St 1 38 40.813358 -73.956461 In Service 1 37 Amsterdam Ave & W 125 St      False 2019-09-06 03:39:45 PM 
Inserting Station: 3542 Amsterdam Ave & W 119 St 21 29 40.8086249 -73.9596214 In Service 1 7 Amsterdam Ave & W 119 St      False 2019-09-06 03:41:50 PM 
Inserting Station: 3543 Morningside Dr & Amsterdam Ave 35 35 40.81028506306003 -73.95736455917358 In Service 1 0 Morningside Dr & Amsterdam Ave      False 2019-09-06 03:42:03 PM 
Inserting Station: 3544 Underhill Ave & Pacific St 11 11 40.

Inserting Station: 3715 Driggs Ave & N 9 St 5 31 40.71816969454208 -73.95520135760307 In Service 1 25 Driggs Ave & N 9 St      False 2019-09-06 03:39:34 PM 
Inserting Station: 3716 40 Ave & Crescent St 23 23 40.753599202005596 -73.93795609474182 In Service 1 0 40 Ave & Crescent St      False 2019-09-06 03:38:56 PM 
Inserting Station: 3718 E 11 St & Avenue B 31 46 40.72746420692392 -73.97950418293476 In Service 1 13 E 11 St & Avenue B      False 2019-09-06 03:39:29 PM 
Inserting Station: 3720 Wyckoff St & Bond St 0 23 40.68461653897949 -73.98721285164356 In Service 1 23 Wyckoff St & Bond St      False 2019-09-06 03:41:13 PM 
Inserting Station: 3721 31 St & Newtown Ave 1 35 40.7675487799971 -73.9209334552288 In Service 1 34 31 St & Newtown Ave      False 2019-09-06 03:40:36 PM 
Inserting Station: 3723 Cadman Plaza E & Johnson St 25 30 40.695317277669034 -73.99015724658966 In Service 1 5 Cadman Plaza E & Johnson St      False 2019-09-06 03:39:59 PM 
Inserting Station: 3724 7 Ave & Central

KeyboardInterrupt: 

---

## I hope this has helped you find your coding legs! Next week we'll get back to descriptive analytics using Python and Pandas. For now, take time to refresh yourself on the content covered in "Introduction to Programming". 

## If you need a referesher on your SQL skills, check out the "Supplementary Info" directory in the class repo.

In [23]:
# using MySQL